In [1]:
# TODO:
# unittest

In [2]:
# Imports
import ast
import bs4
import copy
import pandas
import requests

In [3]:
yelp_alias = 'degustation-new-york'

### Get data

In [4]:
# Set up soup
url = 'https://www.yelp.com/biz/{}'.format(yelp_alias)
r = requests.get(url, timeout=5)
soup = bs4.BeautifulSoup(r.text, 'lxml')

In [5]:
yelp_biz_id = soup.find('meta', {'name': 'yelp-biz-id'})['content']

In [6]:
is_closed = 'CLOSED' in soup.title.text

In [7]:
business_name = soup.find('h1', {'class': 'biz-page-title'}).text.strip()

In [8]:
rating = soup.find('div', {'class': 'i-stars'})
rating = rating['title'].split()[0]
rating = float(rating)

In [9]:
review_count = soup.find('span', {'class': 'review-count rating-qualifier'}).text
review_count = review_count.strip().split()[0]
review_count = int(review_count)

In [10]:
# TODO: store as JSON in PSQL?
ratings_history = soup.find('div', {'id': 'rating-details-modal-content'})
ratings_history = ratings_history['data-monthly-ratings']
ratings_history = ast.literal_eval(ratings_history)
ratings_history = {int(k):v for k,v in ratings_history.items()}

In [11]:
price_range = soup.find('span', {'class': 'business-attribute price-range'}).text

In [12]:
category_list = soup.find('span', {'class': 'category-str-list'}).contents
category_list = [i.string for i in category_list]
category_list = [i for i in category_list if '\n' not in i]

In [13]:
neighborhood_list = soup.find('span', {'class': 'neighborhood-str-list'}).text
neighborhood_list = neighborhood_list.strip().split(', ')

In [14]:
address = soup.find('strong', {'class': 'street-address'})
address = address.find('address')
address = copy.copy(address)    # Make a copy to prevent modifying the original document
address.find('br').replace_with('\n')
address = address.text.strip().split('\n')
address = ', '.join(address)